<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/QSTK_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy==1.6.1

    100% |████████████████████████████████| 13.6MB 1.5MB/s 
keras-preprocessing 1.0.5 has requirement numpy>=1.9.1, but you'll have numpy 1.6.1 which is incompatible.
tensorflow-hub 0.2.0 has requirement numpy>=1.12.0, but you'll have numpy 1.6.1 which is incompatible.
pystan 2.18.1.0 has requirement numpy>=1.7, but you'll have numpy 1.6.1 which is incompatible.
pywavelets 1.0.1 has requirement numpy>=1.9.1, but you'll have numpy 1.6.1 which is incompatible.
fbprophet 0.4.post1 has requirement numpy>=1.10.0, but you'll have numpy 1.6.1 which is incompatible.
xarray 0.11.2 has requirement numpy>=1.12, but you'll have numpy 1.6.1 which is incompatible.
osqp 0.5.0 has requirement numpy>=1.7, but you'll have numpy 1.6.1 which is incompatible.
tensorboard 1.12.2 has requirement numpy>=1.12.0, but you'll have numpy 1.6.1 which is incompatible.
librosa 0.6.2 has requirement numpy>=1.8.0, but you'll have numpy 1.6.1 which is incompatible.
numexpr 2.6.9 has requirement numpy>=1.7, but you'll ha

In [2]:
!pip install scipy

    100% |████████████████████████████████| 17.0MB 1.4MB/s 
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
astropy 2.0.11 has requirement pytest<3.7,>=2.8, but you'll have pytest 3.10.1 which is incompatible.
pymc3 3.6 has requirement joblib<0.13.0, but you'll have joblib 0.13.1 which is incompatible.
thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.0 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
  Found existing installation: numpy 1.6.1
    Uninstalling numpy-1.6.1:
      Successfully uninstalled numpy-1.6.1
  [numpy]
You must restart the runtime in order to use newly installed versions.


In [8]:
!pip install matplotlib

    100% |████████████████████████████████| 235kB 4.2MB/s 
qstk 0.2.8 has requirement python-dateutil==1.5, but you'll have python-dateutil 2.7.5 which is incompatible.
  Found existing installation: python-dateutil 1.5
    Uninstalling python-dateutil-1.5:
      Successfully uninstalled python-dateutil-1.5
  [dateutil]
You must restart the runtime in order to use newly installed versions.


In [9]:
!pip install pandas


In [10]:
!pip install qstk


bokeh 1.0.4 has requirement python-dateutil>=2.1, but you'll have python-dateutil 1.5 which is incompatible.
botocore 1.12.78 has requirement python-dateutil<3.0.0,>=2.1; python_version >= "2.7", but you'll have python-dateutil 1.5 which is incompatible.
matplotlib 2.2.3 has requirement python-dateutil>=2.1, but you'll have python-dateutil 1.5 which is incompatible.
jupyter-client 5.2.4 has requirement python-dateutil>=2.1, but you'll have python-dateutil 1.5 which is incompatible.
  Found existing installation: python-dateutil 2.7.5
    Uninstalling python-dateutil-2.7.5:
      Successfully uninstalled python-dateutil-2.7.5
  [dateutil]
You must restart the runtime in order to use newly installed versions.


In [11]:
!pip install pandas_datareader

In [0]:
# Importing  required packages
import pandas  as pd
pd.TimeSeries = pd.Series

import QSTK.qstkutil.qsdateutil as du
import QSTK.qstkutil.tsutil as tsu
import QSTK.qstkutil.DataAccess as da

import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd

<h2> Computing Sharpe Ratio and Returns </h2>

In [0]:
import logging
logger = logging.getLogger(__name__)
from math import sqrt

def get_prices(ls_symbols, dt_start, dt_end):
    dt_timeofday = dt.timedelta(hours=16)
    ldt_timestamps = du.getNYSEdays(dt_start, dt_end, dt_timeofday)
    #c_dataobj = da.DataAccess('Yahoo')
    c_dataobj = da.DataAccess('Yahoo', cachestalltime=0)
    ls_keys = ['close']
    ldf_data = c_dataobj.get_data(ldt_timestamps, ls_symbols, ls_keys)
    return  dict(zip(ls_keys, ldf_data)) # each field is returned as a single dataframe

def compute_daily_returns(prices):
    na_rets = prices.copy()
    return tsu.returnize0(na_rets)

def compute_standard_deviations(daily_returns):
    return np.std(daily_returns)

def normalize_prices(prices):
    return prices / prices[0,:]

def compute_sharpe_ratio(daily_returns, standard_dev):
    
    avg = np.average(daily_returns)# / len(daily_returns)
    return (sqrt(252) * avg) / standard_dev


In [3]:
import pandas as pd
from pandas.tseries.offsets import BDay
import pandas_datareader.data as dr
import numpy as np
from datetime import datetime


def get_closes(start, end, symbol):
  return dr.DataReader(symbol, 'iex', start, end)['close']


def compute_statistics(symbol, prices):
  #prices = get_closes(start, end, symbol)
  daily_returns = compute_daily_returns(prices)
  std_dev  = compute_standard_deviations(daily_returns)
  sharpe = compute_sharpe_ratio(daily_returns, std_dev)
  total_returns = (prices[-1]*1.0 / prices[0]) -1
  
  
  return {'std_dev' : std_dev,
         'sharpe_ratio' : sharpe,
         'total_returns' : total_returns,
         'symbol': symbol}
  

end = pd.Timestamp(datetime.now())
start = end - BDay(250)
symbols = ['MARA','DPW','TOPS','HAON','HTBX','IMNP','OPGN','SUZBY','ARCO','TRQ','ARTX','PDLI','ENPH','PLPL',
           'DNN','ZYNE','EYEG','FLKS','APHB','PHBI','ENDV']

all_dicts = []
for sym in symbols:
  try:
    all_dicts.append((sym, get_closes(start, end, sym)))#compute_statistics(start, end ,sym)) #compute_statistics(sym, start, end))
  except Exception as e:
    print('Unable to fetch statistics for %s:%s' % (sym, str(e)))




Unable to fetch statistics for HAON:'HAON'
Unable to fetch statistics for SUZBY:'SUZBY'
Unable to fetch statistics for PLPL:'PLPL'
Unable to fetch statistics for PHBI:'PHBI'
Unable to fetch statistics for ENDV:'ENDV'


<h2>Computing Statistics </h2>

In [4]:
    
all_df = []
for symbol, prices in all_dicts:
  all_df.append(compute_statistics(symbol, prices))
    
    

df = pd.DataFrame(all_df)[['symbol', 'total_returns','std_dev', 'sharpe_ratio']]
df



,symbol,total_returns,std_dev,sharpe_ratio
0,MARA,-0.840117,0.072510,-1.143323
1,DPW,-0.936306,0.064622,-2.309098
2,TOPS,-0.516515,0.078360,-0.033539
3,HTBX,-0.501873,0.107234,0.298117
4,IMNP,-0.854929,0.077182,-2.578374
5,OPGN,-0.319009,0.051002,-0.096822
6,ARCO,-0.108906,0.022691,-0.155033
7,TRQ,-0.436823,0.024026,-1.391064
8,ARTX,-0.048571,0.025576,0.075221
9,PDLI,0.297071,0.027180,0.842863


<h2> Computng Future Earnings</h2>

In [5]:
import urllib
import json


def get_future_earnings_df(symbol):

  base_url = 'https://us-central1-datascience-projects.cloudfunctions.net/future_earnings/{}'.format(symbol)
  res = urllib.urlopen(base_url)
  json_dict = res.read() #json.load(res)
  try:
    df = pd.read_json(json_dict)

    cons  = df['Consensus'].values
    fisc = df['Fiscal'].values

    print('Cons:{}, Fisc:{}'.format(type(cons), type(fisc)))

    converted = pd.DataFrame([cons], columns=fisc)
    converted['symbol'] = symbol
    return converted
  except Exception:
    print('exception in retrieving earnings for {}'.format(symbol))

# Get earning estimates for all of them and find common columns
# We need to repeat this process for every stock. And then save as CSV and reduce to common columns
symbols = ['MARA','DPW','TOPS','HAON','HTBX','IMNP','OPGN','SUZBY','ARCO','TRQ','ARTX','PDLI','ENPH','PLPL',
           'DNN','ZYNE','EYEG','FLKS','APHB','PHBI','ENDV']
all_dfs = [get_future_earnings_df(symbol) for symbol in symbols]






exception in retrieving earnings for MARA
exception in retrieving earnings for DPW
exception in retrieving earnings for TOPS
exception in retrieving earnings for HAON
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
exception in retrieving earnings for IMNP
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
exception in retrieving earnings for SUZBY
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
exception in retrieving earnings for TRQ
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
exception in retrieving earnings for PLPL
exception in retrieving earnings for DNN
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
Cons:<type 'numpy.ndarray'>, Fisc:<type 'numpy.ndarray'>
exception in retrieving earnings for FLKS
exception in retrieving earnings for APHB
exception in retrieving earnings for PHBI
exception in retrieving earnings for ENDV


In [6]:
good_ones = filter(lambda df: df is not None, all_dfs)

min_shape = min(map(lambda df: df.shape[1], good_ones))
col_to_project = good_ones[0].columns[0:min_shape]

print ('Cols to project:{}'.format(col_to_project))


reduced_df = reduce(lambda acc, item: item if acc is None else acc.append(item), good_ones, None)

reduced_df


Cols to project:Index([u'Dec 2018', u'Dec 2019', u'Dec 2020'], dtype='object')


,Dec 2018,Dec 2019,Dec 2020,Dec 2021,symbol
0,-1.10,-0.65,-1.15,1.73,HTBX
0,-2.17,-0.93,NaN,NaN,OPGN
0,0.28,0.42,NaN,NaN,ARCO
0,0.15,0.20,0.26,NaN,ARTX
0,-0.51,0.13,0.20,NaN,PDLI
0,-0.14,0.13,0.22,0.39,ENPH
0,-2.87,-2.43,-2.60,-1.19,ZYNE
0,-0.31,-0.27,-0.26,NaN,EYEG


In [0]:
merged= pd.merge(df, reduced_df, how='left', on='symbol')

merged

# SAving to csv file
merged.to_csv('/tmp/results.csv')




In [8]:
# checking the file is there
!cat /tmp/results.csv

,symbol,total_returns,std_dev,sharpe_ratio,Dec 2018,Dec 2019,Dec 2020,Dec 2021
0,MARA,-0.8401171875,0.0725101291844523,-1.1433231706174947,,,,
1,DPW,-0.9363057324840764,0.06462200479311135,-2.3090978032687732,,,,
2,TOPS,-0.5165150789851604,0.07836030090433346,-0.0335387683314553,,,,
3,HTBX,-0.5018726591760299,0.10723411715376657,0.2981174781266307,-1.1,-0.65,-1.15,1.73
4,IMNP,-0.8549290141971606,0.0771819353512611,-2.5783738467237844,,,,
5,OPGN,-0.3190086206896551,0.05100224067362572,-0.09682198998222241,-2.17,-0.93,,
6,ARCO,-0.10890560802518545,0.022690515923983755,-0.15503346605648313,0.28,0.42,,
7,TRQ,-0.4368231046931408,0.02402636700576219,-1.391063695511287,,,,
8,ARTX,-0.0485714285714286,0.025575539757631372,0.07522148278457004,0.15,0.2,0.26,
9,PDLI,0.2970711297071129,0.02718021434039187,0.8428634375496403,-0.51,0.13,0.2,
10,ENPH,1.8552631578947372,0.05125302266418206,1.747077289987807,-0.14,0.13,0.22,0.39
11,DNN,0.08951175406871603,0.03280606013389643,0.43170875253846946,,,,
12,Z

<h2> computing moving averages </h2>

In [0]:
# Authenticating to Google so we can copy over the stuff
from google.colab import auth
auth.authenticate_user()

In [10]:
!gsutil cp /tmp/results.csv gs://datascience-bucket-mm

Copying file:///tmp/results.csv [Content-Type=text/csv]...
/ [1 files][  1.3 KiB/  1.3 KiB]                                                
Operation completed over 1 objects/1.3 KiB.                                      


In [0]:
from datetime import date
end = pd.Timestamp(datetime.now())
start = end - BDay(250)
import numpy as np


def get_data(start, end, symbol):
  return dr.DataReader(symbol, 'iex', start, end)['close']


def compute_avg(prices):
  mapped = map(lambda tpl: tpl[1], prices)
  return np.average(mapped) if prices else 0.0


def compute_moving_avg(prices, days):
  res = [(prices[i][0], prices[i][1],  compute_avg(prices[i-days:i])) for i in range(0, len(prices))]
  return res
  
  

end = pd.Timestamp(datetime.now())
start = end - BDay(400)  
res =get_data(start, end, 'AMZN')  

all_data = zip(res.index, res)

result = compute_moving_avg(all_data, 200)

dr.DataReader('AMZN', 'iex', date(2019,1,14), date(2019,1,23))
  

